# 과제 2. 나만의 데이터 셋 만들기
기계 학습 분야에서 중요한 것 중 하나가 학습에 사용할 자료를 수집하고 관리하는 것입니다.
예를 들어 주어진 문장이 부정인지 긍정인지 판별하는 모델을 학습 시키기 위해서는 긍정 부정이 라벨링된 문장 자료가 필요합니다.

문장 | 긍부정 
-------|--------
진짜 좋은 영화. 가히 올해의 영화라고 불릴만하다. | 긍정 
이런 영화에 100억원 이상 때려 넣은 감독은 충무로에서 쫓겨나야한다. | 부정 

그러나 우리가 이러한 자료를 직접 수집 및 라벨링하기는 쉽지 않다.

한편, 내부 연구나 교육적 목적으로 이미 가공된 인터넷의 글들을 수집하는 것은 공정이용으로 저작권법에 어긋나지 않습니다.
따라서 네이버 영화 리뷰를 크롤링하여 나만의 긍부정 문장 데이터 셋을 만들어 봅시다.


## 2-1. 네이버 영화 리뷰 크롤링 (40점)
현재 상영 중인 영화에 대한 영화 한 줄 평은 다음 링크에서 확인 가능합니다.

[https://movie.naver.com/movie/point/af/list.naver?&page=1](https://movie.naver.com/movie/point/af/list.naver?&page=1)

네이버 영화 한줄평의 주소는 다음과 같이 `<페이지 숫자>`에 해당되는 숫자를 넣으면 해당 페이지를 불러오는 것이 가능합니다.

https://movie.naver.com/movie/point/af/list.naver?&page= `<페이지 숫자>`

이러한 접근을 통해 영화 한 줄 평을 끍어모아 다음과 같은 형태의 `CSV` 파일로 저장하는 코드를 만들어 봅시다.

movie | sentence | score
------|------|------
모가디슈 | 재미잇게 잘봤습니다. 배우들 연기에 더몰입감있게 봤습니다. | 10
낙원의 밤 | 차승원의 유머만 남은 흉내 낸 누아르, 결말조차 완벽하게 폭망 | 2
국화꽃 향기 | 뭔가 와닿지 않는 감동 그리고 스토리. 아쉽지만 여운이 남지 않는다 | 4
미쓰 홍당무 | 서우가 아주 매력적으로 나오는 영화 | 8
더 수어사이드 스쿼드 | 후련하게 봤어요 재밌네요ㅋㅋㅋ잔인하고 징그러운거 좋아하시는 분들께 추천 | 9

### 크롤러를 만들때 다음과 같은 사항을 준수해야 합니다.
1. 실제 학습을 진행할 것은 아니기 때문에 문장은 1000개 정도만 가져오면 충분합니다.
2. 각 페이지를 조회하실때 시간차가 없다면 DDOS 공격으로 분류되어 페이지 접속이 막힐 수가 있습니다. 페이지 조회시 최소 0.5초의 시간 차를 둡시다.
3. 신고되어 삭제된 한줄평은 데이터에 들어가면 안됩니다. 또한 한줄평이 아예 없는 경우도 제외하셔야 합니다.
4. 사용 가능한 라이브러리는 다음과 같습니다.
    * 파이썬 표준 라이브러리
    * Request
    * Beutiful Soup
    * (For colab user) Colab
    * (Optional) Scrapy
5. `CSV` 파일로 저장하실때 Unicode로 저장하셔야 하며, Excel에서 문제 없이 열 수 있어야 합니다.
6. `CSV` 파일은 쉼표로 구분되어야 합니다.
7. `CSV` 파일의 레코드 형식은 다음과 같습니다.
  * movie - 영화 이름이 들어갑니다. 문자열입니다.
  * sentence - 한줄평이 들어갑니다. 문자열입니다.
  * score - 별점이 들어갑니다. 숫자입니다.
8. 크롤링 코드를 실행하여 크롤링을 한 번에 완료해야 합니다.
9. 그외 상세한 구현 방법은 자율입니다.

### 과제를 제출하실때 다음을 제출해야합니다.
1. 크롤링된 데이터를 저장한 `samples.csv`
2. 크롤링 코드


---

# 네이버 영화 리뷰 크롤링 (이창석)

---

### Step 1. 네이버 영화 리뷰 데이터 크롤링.

In [3]:
import time, random
from tqdm import tqdm

import requests
from urllib import parse
from bs4 import BeautifulSoup

base_url = 'https://movie.naver.com/movie/point/af/list.naver?&page={}'

data_list = []
for page in tqdm(range(1,111)):
    url = base_url.format(page)
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    titles = soup.select('tr > td.title')
    for title in titles:
        movie = title.select_one('a.movie').text.strip()
        sentence = title.select_one('br').next_sibling.strip()
        if not sentence:
            continue
        score = title.select_one('div.list_netizen_score > em').text.strip()
        data_list.append((str(movie),str(sentence),int(score)))
    interval = round(random.uniform(0.2,1.2),2)
    time.sleep(interval)


100%|█████████████████████████████████████████| 110/110 [01:39<00:00,  1.10it/s]


### Step 2. 크롤링 된 데이터 확인.

In [14]:
import csv

fields = ['movie', 'sentence', 'score']

with open('samples.csv', 'w',encoding = 'utf-8-sig') as f:
    write = csv.writer(f, delimiter=',',
                      quotechar='"',
                      quoting = csv.QUOTE_MINIMAL)
    write.writerow(fields)
    write.writerows(data_list)


with open('samples.csv', 'r',encoding = 'utf-8-sig') as f:
    reader = csv.reader(f, delimiter=',',
                      quotechar='"',
                      quoting = csv.QUOTE_MINIMAL)
    
    for row in reader:
        print(row)

['movie', 'sentence', 'score']
['스파이더맨: 노 웨이 홈', '시리즈가 주는 감동이 어마어마하다ㅠ', '10']
['미스 슬로운', '신념 있는 로비스트는 자신의 승리만을 믿지 않는다.', '7']
['퍼스트 카우', '마지막 급박한 순간조차도 비로소 둘이 함께 일 때 가장 편안하고 안락함을 느꼈던 두 사람의 우정.', '8']
['공작조: 현애지상', '암살을 눈탱이질한 후에 영감을 받았다며 일을 벌인 장이모우의 최후...태극기 휘날리며 눈탱이질한 집결호 만큼이나 뜬금없다...', '6']
['용쟁호투', '이소룡 몸과 표정을 보고 있으면 시간이 후딱 지나감.', '9']
['크리스마스로 불리는 소년', '쥐가 캐리한 영화 ㅋㅋㅋㅋ', '10']
['우아한 거짓말', '원작도 재밌게 읽었는데 원작만큼 재밌었어요', '10']
['스플릿', '두꺼비는 내 인생 최고의 악역이다ㅋ아직도 줘패주고싶네 두꺼비새끼 (칭찬임)', '10']
['씽2게더', '마지막 장면 30분은 그냥 최고... 꼭 영화관에서 보세요', '8']
['안시성', '병두야 연기 때려치자 그냥', '1']
['경관의 피', '보면서 물음표가 많이 나오는 영화였어요 뭔가 말하고 싶은것은 많는데 제대로된 메시지는 없는 난잡한 스토리였어요', '6']
['제인 에어', '마음이 간드러지는 영화', '8']
['소녀의 세계', '영상미가 너무좋네요 뿜뿜', '10']
['낙원의 밤', '잠깐 나온 이문식 연기력만 10점 준다 ㅋㅋㅋㅋ진짜 남는거 아무것도 없이 기분만 더러운 영화', '1']
['자산어보', '왜 인간은 인간이 갈길을 모르는걸까', '6']
['경관의 피', '쓸대없이 여배우 안쓰고 간만에 재밌게본 한국영화', '8']
['스파이더맨: 노 웨이 홈', '코로나 장기화로 지친 시기에 만나는 역대급 블록버스터', '10']
['경관의 피', '재밋어요 지루하지않아요!!!', '10']
['씽2게더', '최고~개인적으로 1편보다 좋았음', '10']
['둠스데이 -

## 2-2. 네이버 영화 데이터 셋 제작 (30점)
`CSV` 파일로 저장된 데이터를 쉽게 접근하기 위해서는 파이썬 내에서 구조화할 필요가 있습니다.
이를 위하여 데이터에 접근 가능한 `RawMovieReview` 클래스를 만들어 봅시다.


### `RawMovieReview` 클래스는 다음을 준수해야 합니다.
0. 이하 모든 클래스 공통 사항
    * 클래스 속성은 사용이 금지됩니다.
    * 전역 변수 사용이 금지됩니다.
1. 생성자
    * 생성자의 인자는 `str`타입의 `file_name` 하나만을 받습니다.
    * `file_name` 인자의 기본값은 `"samples.csv"`입니다.
    * 해당 객체는 해당 파일을 다루는 객체가 되어야 합니다.
    * 모든 속성 값은 `protected` 홋은 `private`이어야 합니다. 즉, 메소드만 `public`입니다.
2. Indexing
    * 대괄호로 N번째 sample에 접근할 수 있어야 합니다.
    * N은 0부터 시작합니다. 즉, `dataset[0]`은 첫번째 한줄 평 입니다.
    * Indexing 결과값은 `영화 이름, 한줄평, 점수`로 타입은 `(str, str, int)` 형태의 튜플입니다.
    * 대괄호로 읽기만 가능하고 수정은 불가능해야 합니다.
3. Length
    * `len(dataset)`의 형태로 데이터셋 내의 한줄평 개수를 조회할 수 있어야 합니다.

### 과제를 제출하실때 다음을 제출해야합니다.
1. `RawMoviewReview` 클래스가 정의된 코드 (`.py` 혹은 `.ipynb`)


In [17]:
class RawMovieReview:
    def __init__(self, fileName: str = 'samples.csv'):
        self._fileName = fileName
        self._data = []
        
        with open(self._fileName, 'r',encoding = 'utf-8-sig') as f:
            reader = csv.reader(f, delimiter=',',
                      quotechar='"',
                      quoting = csv.QUOTE_MINIMAL)
            for row in reader:
                self._data.append(row)
        self._data = self._data[1:] # remove columns (movie, sentence, score)
        
        
    def __len__(self):
        return len(self._data)
    
    def __getitem__(self, index):
        self._title = str(self._data[index][0])
        self._sentence = str(self._data[index][1])
        self._score = int(self._data[index][2])
        return tuple([self._title, self._sentence, self._score])

In [21]:
raw_data = RawMovieReview()
print(f"데이터 타입 : {type(raw_data[100])}") # 튜플 데이터 타입.
print(f"한줄 평 : {raw_data[100]}") # 101번째 한줄 평.
print(f"데이터 한줄 평 개수 : {len(raw_data)}") # 데이터 셋 내의 한줄 평 개수 조회.
print(type(raw_data[100][0]))
print(type(raw_data[100][1]))
print(type(raw_data[100][2]))

데이터 타입 : <class 'tuple'>
한줄 평 : ('위시 드래곤', '재미는 있으나, 구성은 알라딘', 6)
데이터 한줄 평 개수 : 1014
<class 'str'>
<class 'str'>
<class 'int'>


## 2-3. 네이버 영화 학습 데이터 셋 제작 (30점)
위에서 제작한 데이터 셋은 `CSV` 파일을 모두 조회하지만, 학습에 바로 사용되기에는 다소 불편합니다.
따라서 이를 상속한 `MovieReview`를 만들어 봅시다. 

### `MovieReview` 클래스는 다음을 준수해야 합니다.
1. 상속
    * 위에서 정의한 `RawMoviewReview` 클래스를 상속받아야합니다.
    * `RawMoviewReview` 내의 데이터를 복사하면 안됩니다.
    * 즉, 저장된 `CSV`파일은 부모 클래스의 속성 혹은 메소드로 접근해야합니다.
2. 생성자
    * 생성자의 인자는 부모의 인자와 `int`타입의 `score_threadhold`를 받습니다.
    * 해당 클래스에는 `CSV`파일 및 그 내용이 속성으로 들어가면 안됩니다.
    * 즉, 영화 데이터는 부모에만 저장되어야합니다.
3. Indexing
    * 부모의 Indexing을 재정의합니다. (Overriding)
    * 기본적인 기능은 부모의 기능과 비슷하나 이하의 사항이 다릅니다.
    * Indexing 결과 값은 `한줄평, 긍부정`으로 타입은 `(str, bool)` 형태의 튜플입니다.
    * 점수가 객체의 `score_threadhold` 이상일 경우 긍정이 `True`, 미만이면 `False` 입니다.
    * 한줄평 string의 경우 양쪽에 공백이 없어야 합니다.

### 과제를 제출하실때 다음을 제출해야합니다.
1. `MoviewReview` 클래스가 정의된 코드 (`.py` 혹은 `.ipynb`)

In [22]:
class MovieReview(RawMovieReview):
    def __init__(self, fileName: str = 'samples.csv', ScoreThr: int = 5): # Score thr 1 ~ 10
        super().__init__()
        self._ScoreThr = ScoreThr
        
    def __getitem__(self, index):
        if int(super().__getitem__(index)[2]) >= self._ScoreThr:
            return str(super().__getitem__(index)[1]), True
        else:
            return str(super().__getitem__(index)[1]), False

In [23]:
review = MovieReview()
for i in range(1,100):
    print(review[i])

('신념 있는 로비스트는 자신의 승리만을 믿지 않는다.', True)
('마지막 급박한 순간조차도 비로소 둘이 함께 일 때 가장 편안하고 안락함을 느꼈던 두 사람의 우정.', True)
('암살을 눈탱이질한 후에 영감을 받았다며 일을 벌인 장이모우의 최후...태극기 휘날리며 눈탱이질한 집결호 만큼이나 뜬금없다...', True)
('이소룡 몸과 표정을 보고 있으면 시간이 후딱 지나감.', True)
('쥐가 캐리한 영화 ㅋㅋㅋㅋ', True)
('원작도 재밌게 읽었는데 원작만큼 재밌었어요', True)
('두꺼비는 내 인생 최고의 악역이다ㅋ아직도 줘패주고싶네 두꺼비새끼 (칭찬임)', True)
('마지막 장면 30분은 그냥 최고... 꼭 영화관에서 보세요', True)
('병두야 연기 때려치자 그냥', False)
('보면서 물음표가 많이 나오는 영화였어요 뭔가 말하고 싶은것은 많는데 제대로된 메시지는 없는 난잡한 스토리였어요', True)
('마음이 간드러지는 영화', True)
('영상미가 너무좋네요 뿜뿜', True)
('잠깐 나온 이문식 연기력만 10점 준다 ㅋㅋㅋㅋ진짜 남는거 아무것도 없이 기분만 더러운 영화', False)
('왜 인간은 인간이 갈길을 모르는걸까', True)
('쓸대없이 여배우 안쓰고 간만에 재밌게본 한국영화', True)
('코로나 장기화로 지친 시기에 만나는 역대급 블록버스터', True)
('재밋어요 지루하지않아요!!!', True)
('최고~개인적으로 1편보다 좋았음', True)
('이게 바로 진짜 헐리우드 B+급이지 ㅋㅋㅋㅋ', False)
('아 그때 그 추억. 짜릿한 등장.', True)
('소름이돋는 연기… 지금의 저 배우들을 보면 정말저 시대와 같은 분들이었을까라는의심에 더 소름이 돋는다…배우는배우다!!!', True)
('액션도 스토리도 지금까지 시리즈 중 최고였어요 ㅠㅠ 꼭보세여', True)
('옛 스파이더 주인공들과 현 스파이더맨 한 화면에 나올줄이야.. 정말 상상도 못했습니다. 어린시절 추억들과 지금의 감

## 최종 제출
* 모든 파일을 `zip`으로 압축하여 `HW2_<한글 이름>.zip` 형태로 제출합니다.


## 채점 기준 표
각각의 문제에 대해서 이하의 감점이 각각 들어갈 수 있습니다. 음수 점수는 없습니다.
아래의 채점 기준표는 확정이 아니며, 추후 수정된 점수 기준이 적용될 수 있습니다.

감점요소 | 설명 | 감점 점수
-----|-----|-----
API 문서와 다른 입출력 | 과제 내 명시된 API 및 포멧과 입출력이 다릅니다. | -15
비효율적인 시간복잡도 | 시간복잡도가 최선이 아닙니다. (크롤링 $O(N)$, Indexing $O(1)$)| -10
비효율적인 공간 구조 | 부모 자식간 데이터가 중복하는 형태로 클래스가 작성되어 있습니다. | -10
명시되어 있지 않은 외부 라이브러리 사용 | 지정된 라이브러리 외에 다른 라이브러리를 사용하였습니다 | -10
권장하지 않는 문법 | 전역 변수나 클래스 속성을 사용하였습니다. | -10